In [ ]:
import import_ipynb
import pickle
import numpy as np
from matplotlib import pyplot as plt
import arc_data
from copy import deepcopy as dc

In [ ]:
import machine

In [ ]:
class Functions():
    def __init__(self):
        self.functionD={}
    def add(self,f,name,kind):
        self.functionD[name]={}
        self.functionD[name]['func']=f
        self.functionD[name]['kind']=kind
    def get(self,name):
        if name in list(self.functionD.keys()):
            return self.functionD[name]['func']
        else: return name
        return 
    def ismem(self,name):
        if self.functionD[name]['kind']=='mem': return True
        else: return False

In [ ]:
libF=Functions()

In [ ]:
def log(func,args,stage):
    global m,logL,counter,attempt
    logD={'func':func,
          'args':args,
          'stage':stage,
          'memory':dc(m.memory),
          'output':dc(m.output),
          'counter':counter,
          'attempt':attempt}
    counter+=1
    logL+=[logD]
    return

In [ ]:
def get_objects(embed):
    global m,logL
    memsave=m.memory
    auxsave=m.aux
    outsave=m.output
    log('get_objects',(embed),'before')
    try:
        embed=str(embed)
        objectsL,offsetsL=machine.objects(m.memory[0],machine.member,oarg='all',embed=embed)
        m.memory=objectsL
        m.aux=offsetsL
        log('get_objects',(embed),'success')
        return True
    except:
        m.memory=memsave
        m.aux=auxsave
        m.output=outsave
        log('get_objects',(embed),'fail')
        return False
get_objects.arity=1
libF.add(get_objects,'get_objects','util')

In [ ]:
def assemble_output(embed):
    global m,logL
    memsave=m.memory
    auxsave=m.aux
    outsave=m.output
    log('assemble_output',(embed),'before')
    try:
        embed=str(embed)
        objectsL=m.output
        offsetsL=m.aux
        Nr,Nc=len(m.input[0]),len(m.input[0][0])
        m.output=machine.assemble_grid(objectsL,offsetsL,Nr,Nc,embed=embed,scale=1)
        #m.memory=m.input
        log('assemble_output',(embed),'success')
        return True
    except:
        m.memory=memsave
        m.aux=auxsave
        m.output=outsave
        log('assemble_output',(embed),'fail')
        return False
assemble_output.arity=1
libF.add(assemble_output,'assemble_output','util')

In [ ]:
def applyfn(fname,arg):
    global m,logL,libF
    memsave=m.memory
    auxsave=m.aux
    outsave=m.output
    fname=str(fname)
    arg=str(arg)
    log('apply '+fname,arg,'before')
    try:
        func=libF.get(fname)
        argf=libF.get(arg)
        kind=libF.functionD[fname]['kind']
        if kind=='mem':m.apply(func,argf,fname=fname)
        elif kind=='util': func(argf)
        log('apply '+fname,arg,'success')
        return True
    except: 
        m.memory=memsave
        m.aux=auxsave
        m.output=outsave
        log('apply '+fname,arg,'fail')
        return False
applyfn.arity=2

In [ ]:
def applyfn_debug(fname,arg):
    global m,logL,libF
    memsave=m.memory
    auxsave=m.aux
    outsave=m.output
    fname=str(fname)
    arg=str(arg)
    log('apply',arg,'before')
    func=libF.get(fname)
    argf=libF.get(arg)
    kind=libF.functionD[fname]['kind']
    if kind=='mem':m.apply(func,argf,fname=fname)
    elif kind=='util': func(argf)
    log('apply',arg,'success')
    return True
applyfn.arity=2

In [ ]:
def out():
    global m,logL
    log('out',(),'before')
    m.out()
    log('out',(),'success')
    return True
out.arity=0

In [ ]:
def copy():
    log('copy',(),'before')
    global m,logL
    m.copy()
    log('copy',(),'success')
    return True
copy.arity=0

In [ ]:
def equals_target():
    global m,attempt
    log('equals_target',(),'before')
    attempt+=1
    if m.output==m.target: 
        log('equals_target',(),'success')
        return True
    else: 
        m.memory=[]
        m.aux=[]
        m.output=[]
        copy()
        applyfn('get_objects','bbox')
        log('equals_target',(),'fail')
        return False
equals_target.arity=0

In [ ]:
def noop(arg):
    return True
noop.arity=1
libF.add(noop,'noop','util')

Library functions

In [ ]:
size=lambda g: sum([len([1 for i in l if i!=0]) for l in g])
libF.add(size,'size','aux')

In [ ]:
area=lambda g: len(g)*len(g[0])
libF.add(area,'area','aux')

In [ ]:
select=lambda gL,f: [el for el in gL if f(el)==(lambda l,f: max([f(e) for e in l]))(gL,f)][-1]
libF.add(select,'select','mem')

In [ ]:
from arc_data import ARC, FewShotARC

In [ ]:
a=ARC()

In [ ]:
a=FewShotARC()

In [ ]:
a.create_tasks(kind='meta_train')

In [ ]:
a.create_tasks(kind='meta_test')

In [ ]:
#task=a.get_task(4)
task=a.get_task(173)
# task=a.get_task(262,kind='meta_train')

In [ ]:
a.plot_task(task,kind='orig')

In [ ]:
a.plot_task(a.get_fs_task(173))

In [ ]:
grid=a.get_task(262)['train'][0]['input']

In [ ]:
target=a.get_task(262)['train'][0]['output']

In [ ]:
testgrid=a.get_task(262)['test'][0]['input']

In [ ]:
testtarget=a.get_task(262)['test'][0]['output']

In [ ]:
#a.plot_task(a.get_task(9))

In [ ]:
#testgrid=a.get_task(173)['test'][0]['input']

In [ ]:
#testtarget=a.get_task(173)['test'][0]['output']

In [ ]:
#testgrid

In [ ]:
#a.example2img(grid)

In [ ]:
logL=[]
counter,attempt=0,0
m=machine.Machine()
m.debug=True
#m.load(grid,target)
m.load(testgrid,testtarget)

In [ ]:
copy()
applyfn('get_objects','bbox')
applyfn('select','size')
out()
# applyfn('assemble_output','bbox')
# equals_target()

In [ ]:
m.log

In [ ]:
m.input

In [ ]:
m.memory

In [ ]:
m.aux

In [ ]:
m.output

In [ ]:
equals_target()

In [ ]:
target

In [ ]:
logL

Prolog interface to machine m

In [ ]:
from pyswip import Prolog, registerForeign

In [ ]:
p=Prolog()

In [ ]:
for f in p.query("retract(option(X)),fail"): pass

In [ ]:
for f in p.query("retract(function(X)),fail"): pass

In [ ]:
for f in p.query("retract(argument(X,Y)),fail"): pass

In [ ]:
registerForeign(get_objects)
registerForeign(assemble_output)
registerForeign(out)
registerForeign(copy)
registerForeign(applyfn)
registerForeign(equals_target)
registerForeign(noop)

In [ ]:
p.assertz('option(bbox)')

In [ ]:
p.assertz('option(full)')

In [ ]:
p.assertz('function(select)')

In [ ]:
p.assertz('argument(select,size)')

In [ ]:
p.assertz('argument(select,area)')

In [ ]:
#p.assertz('function(get_objects)')

In [ ]:
#p.assertz('argument(get_object,bbox)')

In [ ]:
p.assertz('function(noop)')

In [ ]:
p.assertz('argument(noop,null)')

In [ ]:
p.assertz('function(assemble_output)')

In [ ]:
p.assertz('argument(assemble_output,bbox)')

In [ ]:
logL=[]
counter,attempt=0,0
m=machine.Machine()
m.load(grid,target)

In [ ]:
qstr="copy(),get_objects(bbox),function(F),argument(F,A),applyfn(F,A),out(),applyfn(assemble_output,bbox),equals_target(),!"

In [ ]:
qstr="copy(),get_objects(bbox),function(F),argument(F,A),applyfn(F,A),function(G),argument(G,GA),applyfn(G,GA),out(),function(P),argument(P,PA),applyfn(P,PA),equals_target(),!"

In [ ]:
l=list(p.query(qstr))

In [ ]:
qstr

In [ ]:
l

In [ ]:
equals_target()

In [ ]:
theory=qstr
for k in l[0].keys():
    print(k,l[0][k])
    theory=theory.replace(k,l[0][k])

In [ ]:
theory1='copy(),get_objects(bbox),function(select),argument(select,size),applyfn(select,size),out(),equals_target(),!'

In [ ]:
logL=[]
counter,attempt=0,0
m=machine.Machine()
m.load(grid,target)
# m.load(testgrid,testtarget)

In [ ]:
l=list(p.query(theory1))

In [ ]:
l

In [ ]:
equals_target()

In [ ]:
m.output

In [ ]:
logL

Abduction

In [ ]:
# selectA=lambda gL,i: gL[int(i)]
# libF.add(selectA,'selectA','mem')

In [ ]:
selectA=lambda gL,f: gL[f()]
libF.add(selectA,'selectA','mem')

In [ ]:
logL=[]
counter,attempt=0,0
m=machine.Machine()
m.load(grid,target)
#m.load(testgrid,testtarget)

In [ ]:
m.target

In [ ]:
copy()
applyfn('get_objects','bbox')
#applyfn('selectA','e3')
#out()
#applyfn('assemble_output','bbox')
#equals_target()

In [ ]:
m.memory

In [ ]:
equals_target()

In [ ]:
logL

In [ ]:
echo=lambda x: int(x)

In [ ]:
p.assertz('function(selectA)')

In [ ]:
from functools import partial

In [ ]:
e3=partial(echo,3)
libF.add(e3,'e3','mem')

In [ ]:
e2=partial(echo,2)
libF.add(e2,'e2','mem')

In [ ]:
e1=partial(echo,1)
libF.add(e1,'e1','mem')

In [ ]:
e0=partial(echo,0)
libF.add(e0,'e0','mem')

In [ ]:
for f in p.query("retract(argument(X,Y)),fail"): pass

In [ ]:
p.assertz('argument(selectA,e3)')

In [ ]:
p.assertz('argument(selectA,e2)')

In [ ]:
p.assertz('argument(selectA,e1)')

In [ ]:
p.assertz('argument(selectA,e0)')

In [ ]:
logL=[]
counter,attempt=0,0
m=machine.Machine()
m.load(grid,target)
#m.load(testgrid,testtarget)

In [ ]:
m.debug=True

In [ ]:
qstr="copy(),get_objects(bbox),argument(selectA,A),applyfn(selectA,A),out(),applyfn(assemble_output,bbox),equals_target(),!"

In [ ]:
qstr="copy(),get_objects(bbox),argument(selectA,A),applyfn(selectA,A),out(),equals_target(),!"

In [ ]:
qstr="copy(),get_objects(bbox),function(F),argument(F,A),applyfn(F,A),out(),applyfn(assemble_output,bbox),equals_target(),!"

In [ ]:
list(p.query(qstr))

In [ ]:
logL

In [ ]:
m.output

In [ ]:
selectAL=[lD for lD in logL if lD['func']=='apply selectA']

In [ ]:
attemptL=[(lD['attempt'],lD['stage']) for lD in logL if lD['func']=='equals_target' and lD['stage']!='before']

In [ ]:
attemptL

In [ ]:
[(lD['attempt'],lD['memory'],[a[1] for a in attemptL if a[0]-1==lD['attempt']]) for lD in selectAL if lD['stage']=='before']

In [ ]:
[(lD['attempt'],lD['memory'],[a[1] for a in attemptL if a[0]-1==lD['attempt']]) for lD in selectAL if lD['stage']!='before']

Exeriments

In [ ]:
l=list(p.query("copy(),get_objects(bbox),applyfn(select,size),out()"))

In [ ]:
l=list(p.query("copy(),get_objects(bbox),applyfn(select,size),applyfn(noop,null),out(),applyfn(assemble_output,bbox)"))

In [ ]:
l=list(p.query("copy(),get_objects(bbox),applyfn(noop,null),out(),applyfn(assemble_output,bbox)"))

In [ ]:
l=list(p.query("copy(),get_objects(bbox),function(F),argument(F,A),applyfn(F,A),out(),applyfn(assemble_output,bbox),equals_target(),!"))

In [ ]:
l=list(p.query("copy(),get_objects(bbox),function(F),argument(F,A),applyfn(F,A),out(),applyfn(assemble_output,bbox),equals_target(),!"))


In [ ]:
l=list(p.query("copy(),get_objects(bbox),function(F),argument(F,A),applyfn(F,A),function(G),argument(G,GA),applyfn(G,GA),out(),equals_target(),!"))


In [ ]:
qstr="copy(),get_objects(bbox),function(F),argument(F,A),applyfn(F,A),function(G),applyfn(G,bbox),out(),equals_target(),!"

In [ ]:
qstr="copy(),get_objects(bbox),function(F),argument(F,A),applyfn(F,A),function(G),applyfn(G,bbox),out(),applyfn(assemble_output,bbox),equals_target(),!"

In [ ]:
l=list(p.query(qstr))

In [ ]:
l

In [ ]:
theory=qstr
for k in l[0].keys():
    print(k,l[0][k])
    theory=theory.replace(k,l[0][k])

In [ ]:
theory

In [ ]:
l

In [ ]:
grid

In [ ]:
m.memory

In [ ]:
m.output

In [ ]:
equals_target()

In [ ]:
list(p.query("function(F),argument(F,A)"))

In [ ]:
logL

Lambda operations on grids

In [ ]:
size=lambda g: sum([len([1 for i in l if i!=0]) for l in g])

In [ ]:
area=lambda g: len(g)*len(g[0])

In [ ]:
#maxcomp=lambda l,f: max([f(e) for e in l])

In [ ]:
#maxel=lambda gL,f: [el for el in gL if f(el)==maxcomp(gL,f)][0]

In [ ]:
select=(lambda gL,f: [el for el in gL if f(el)==(lambda l,f: max([f(e) for e in l]))(gL,f)][-1])

In [ ]:
select(m.memory,size)

In [ ]:
[1 for i in grid[0] if i!=0]

In [ ]:
len = λl.if (empty l) 0 (plus 1 (len (tail l)))

In [ ]:
mylen=lambda l: h,*t=l, 1 if t==[] else 10

In [ ]:
h,*t=[1]

In [ ]:
t

In [ ]:
mylen([1])

In [ ]:
Nr,Nc=len(m.input[0]),len(m.input[0][0])

In [ ]:
Nr

In [ ]:
machine.assemble_grid(m.memory,m.aux,Nr,Nc,embed='bbox',scale=1)

In [ ]:
libF.functionD

In [ ]:
generated=["lambda g: len ([ 1 for i in l if i! =0] ) for l in g] ) for l in g] )",
 "lambda g: len ([ 1 for i in l if i! =0] ) for l in g] )",
 "lambda g: len ([ 1 for i in l if i! =0] ) for l in g] )) (gL, f )",
 "lambda g: len ([ 1 for i in l if i! =0] ) for l in g] )) (gL, f) ] [-1 ]",
 "lambda g: len ([ 1 for i in l if i! =0] ) for l in g] )) (gL, f) for l in g] )",
 "lambda g: len ([ 1 for i in l if i! =0] ) for l in g] )] [-1 ]",
 "lambda g: len (g) *len (g [0] ) for l in g] )",
 "lambda g: len (g) *len (g [0] )",
 "lambda g: len (g) *len (g [0] )) (gL, f )",
 "lambda g: len (g) *len (g [0] )) (gL, f) ] [-1 ]",
 "lambda g: len (g) *len (g [0] )) (gL, f) for l in g] )",
 "lambda g: len (g) *len (g [0] )] [-1 ]",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] ) for l in g] ) for l in g] )",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] ) for l in g] )",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] ) for l in g] )) (gL, f )",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] ) for l in g] )) (gL, f) ] [-1 ]",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] ) for l in g] )) (gL, f) for l in g] )",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] ) for l in g] )] [-1 ]",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] )",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] )) (gL, f )",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] )) (gL, f) ] [-1 ]",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] )) (gL, f) for l in g] )",
 "lambda g: sum ([ len ([ 1 for i in l if i! =0] )] [-1 ]",
 "lambda g: sum ([ len (g) *len (g [0] ) for l in g] )",
 "lambda g: sum ([ len (g) *len (g [0] )",
 "lambda g: sum ([ len (g) *len (g [0] )) (gL, f )",
 "lambda g: sum ([ len (g) *len (g [0] )) (gL, f) ] [-1 ]",
 "lambda g: sum ([ len (g) *len (g [0] )) (gL, f) for l in g] )",
 "lambda g: sum ([ len (g) *len (g [0] )] [-1 ]",
 "lambda gL, f: [el for el in gL if f (el) == (lambda l, f: max ([ f (e) for e in l] ) for l in g] )",
 "lambda gL, f: [el for el in gL if f (el) == (lambda l, f: max ([ f (e) for e in l] )",
 "lambda gL, f: [el for el in gL if f (el) == (lambda l, f: max ([ f (e) for e in l] )) (gL, f )",
 "lambda gL, f: [el for el in gL if f (el) == (lambda l, f: max ([ f (e) for e in l] )) (gL, f) ] [-1 ]",
 "lambda gL, f: [el for el in gL if f (el) == (lambda l, f: max ([ f (e) for e in l] )) (gL, f) for l in g] )",
 "lambda gL, f: [el for el in gL if f (el) == (lambda l, f: max ([ f (e) for e in l] )] [-1 ]"]

In [ ]:
exec('('+generated[30]+')(grid)')

In [ ]:
(lambda g: sum([len([1 for i in l if i!=0]) for l in g]))(grid)

In [ ]:
generated[0]

In [ ]:
(lambda g: len ([ 1 for i in l if i! =0] ) for l in g] ) for l in g] ))(grid)

In [ ]:
g=grid

In [ ]:
[len ([ 1 for i in l if i! =0] ) for l in g] ] for l in g]